In [1]:
import openpyxl

In [2]:
%run MyFunctions.ipynb

In [3]:
## Import data
filename = "ResearchInChildAndAd_DATA_2018-12-14_1531.xlsx"
workbook = openpyxl.load_workbook(filename)
sheet1 = workbook["ResearchInChildAndAd_DATA_2018-"]
sheet2 = workbook["Tasks"]
sheet3 = workbook["Illness_Keywords"]

In [4]:
## Make columns for new data
IGNORE_KEYWORDS = ['query', 'vs', 'r/o', 'rule out', 'versus']

sheet3_columns = get_column_names(sheet3)
ILLNESS_KEYWORDS = read_in_illness_keywords(sheet3, sheet3_columns)
ILLNESSES = get_key_list(ILLNESS_KEYWORDS)

## Add columns
set_headers(sheet1, 'admission', ILLNESSES)
set_headers(sheet1, 'discharge', ILLNESSES)

In [5]:
## Create a dictionary of column names
column_names = get_column_names(sheet1)

In [6]:
## Module 1: Set binary values to 0
set_binaries_to_zero(sheet1, "admission", ILLNESSES)
set_binaries_to_zero(sheet1, "discharge", ILLNESSES)

In [7]:
## Module 2: Fill in main diagnosis
set_main_diagnosis(sheet1, "admission", IGNORE_KEYWORDS, ILLNESS_KEYWORDS)
set_main_diagnosis(sheet1, "discharge", IGNORE_KEYWORDS, ILLNESS_KEYWORDS)

***************
['Trauma', 'Disruptive, Impulse-Control, and Conduct']
adjustment disorder with disturbance of conduct
***************
['Neurodevelopmental', 'Intellectual Disability/Global Developmental Delay']
neurodevelopmental delay with intellectual disability
***************
['Schizophrenia Spectrum and Other Psychotic Disorders', 'Trauma']
none recorded - past diagnoses of adjustment disorder and consider diagnoses of psychosis
***************
['Trauma', 'Dissociative']
ptsd with dissociated symptoms
***************
['Schizophrenia Spectrum and Other Psychotic Disorders', 'Trauma']
adjustment disorder secondary to diabetes diagnosis and psychosocial stressors
***************
['Depression', 'Trauma']
adjustment disorder with depressed mood
***************
['Intellectual Disability/Global Developmental Delay', 'ASD']
asd with intellectual disability
***************
['Depression', 'Suicidal']
major depressive episode with suicidal ideation
***************
['Depression', 'Suicidal']

In [8]:
## Module 3: Read Diagnosis, break into substrings, fill in
one_hot_encode_diagnoses(sheet1, 'admission', IGNORE_KEYWORDS, ILLNESS_KEYWORDS)
one_hot_encode_diagnoses(sheet1, 'discharge', IGNORE_KEYWORDS, ILLNESS_KEYWORDS)

In [9]:
## Output file
workbook.save("output-{}.xlsx".format(getTime()))